In [15]:
import json
from dataclasses import dataclass
from pathlib import Path

import pandas as pd
import requests
import tomli

In [16]:
@dataclass
class PredResult:
    area: str = None
    lat: float = None
    lon: float = None
    lat_lon: tuple[float, float] = None
    grid_zone: str = None
    ptid: int = None
    date: str = None
    max_temp: float = None
    min_temp: float = None
    predicted_load: float = None

In [17]:
API_TOML_DIR = Path(Path.cwd().parent, "api_creds.toml")
WEATHER_LOCATION_MAPPING_DIR = Path(
    Path.cwd().parent, "data_information", "weather_location_mapping.json"
)
PTID_AREA_MAPPING_DIR = Path(
    Path.cwd().parent, "data_information", "PTID_name_mapping.json"
)

In [18]:
with open(API_TOML_DIR, "rb") as f:
    key = tomli.load(f)["api_key"]

In [19]:
def get_forecast(
    lat_lon: tuple[float, float], forecast_days: int = 3, api_key: str = key
) -> dict:
    """Gets a forecase for a given lat lon from the https://www.weatherapi.com/ site.
    Requires an api key to be defined.

    Args:
        lat_lon (tuple[float, float]): lat/lon of the location to be forecasted
        forecast_days (int, optional): number of days to forecast, note free tier weatherapi is restricted to 14 days. Defaults to 3.
        api_key (str, optional): api key for weather api. Defaults to key.

    Raises:
        SystemExit: generic error catch for incorrect request parameters

    Returns:
        dict: json of the returned api call#
    """
    BASE_URL = "https://api.weatherapi.com/v1/forecast.json?"
    str_lat_lon = ",".join([str(x) for x in lat_lon])
    query_params = {"q": str_lat_lon, "days": forecast_days, "key": api_key}
    try:
        response = requests.get(BASE_URL, params=query_params)
    except (
        requests.exceptions.RequestException
    ) as e:  # TODO Generic error catching = bad
        raise SystemExit(e)
    return response.json()

In [20]:
def parse_forcast_response(forecast_response_json: dict) -> dict:
    """From a full api response dict, extract the latitude, longitude, forecast dates and corresposnding
    min and max termperatures forecasted,

    Args:
        forecast_response_json (dict): full response from get_forecast function

    Returns:
        dict: latitude, longitude, dates, minmax temp dictionary
    """
    num_days_forecasted = len(forecast_response_json["forecast"]["forecastday"])
    lat = forecast_response_json["location"]["lat"]
    lon = forecast_response_json["location"]["lon"]
    dates = []
    max_temps = []
    min_temps = []
    for day in range(num_days_forecasted):
        dates.append(forecast_response_json["forecast"]["forecastday"][day]["date"])
        max_temps.append(
            forecast_response_json["forecast"]["forecastday"][day]["day"]["maxtemp_c"]
        )
        min_temps.append(
            forecast_response_json["forecast"]["forecastday"][day]["day"]["mintemp_c"]
        )
    return {
        "lat": lat,
        "lon": lon,
        "dates": dates,
        "max_temps": max_temps,
        "min_temps": min_temps,
    }

In [21]:
response = parse_forcast_response(get_forecast(lat_lon=(48.5, 2.35)))

In [22]:
mapping_json = json.loads(
    Path(WEATHER_LOCATION_MAPPING_DIR).read_text(encoding="UTF-8")
)
area_lat_lon = mapping_json["lat_lon"]
grid_zone_mapping = mapping_json["grid_zone"]

In [23]:
ptid_area_mapping = json.loads(Path(PTID_AREA_MAPPING_DIR).read_text(encoding="UTF-8"))

In [24]:
df = pd.DataFrame.from_dict(area_lat_lon).T.rename(columns={0: "Lat", 1: "Lon"})
df["Grid Zone"] = df.index.map(grid_zone_mapping)
df["PTID"] = df["Grid Zone"].map(ptid_area_mapping)
df["Lat_Lon"] = tuple(zip(df["Lat"], df["Lon"]))
df = df.reset_index(drop=False).rename(columns={"index": "Area"})

In [25]:
# Converting dataframe to list of dataclasses
area_info = []  # TODO This is a poor name
for row in df.itertuples():
    result_class = PredResult()
    result_class.area = row.Area
    result_class.lat = float(row.Lat)
    result_class.lon = float(row.Lon)
    result_class.lat_lon = tuple([float(row.Lat), float(row.Lon)])
    result_class.grid_zone = (
        row._4
    )  # Not sure why grid_zone col name hasnt flowed through
    result_class.ptid = float(row.PTID)
    area_info.append(result_class)

In [26]:
def write_forecast_results(
    results_list: list[PredResult], **kwargs
) -> list[PredResult]:
    """Gets min/max temperatures for given number of days as specified by
    "forecast_days" kwarg passed to get_forecast function

    Args:
        results_list (list[PredResult]): list of PredResult dataclasses holding area info:
        area name, lat_lon, grid_zone, ptid

    Returns:
        list[PredResult]: same list of PredResults with additional date and min/max
        forecasted temperature data
    """
    for res in results_list:
        forecast = parse_forcast_response(get_forecast(lat_lon=res.lat_lon, **kwargs))
        res.date = forecast["dates"]
        res.max_temp = forecast["max_temps"]
        res.min_temp = forecast["min_temps"]
    return results_list

In [27]:
forecast_results = write_forecast_results(area_info)

In [28]:
forecast_results

[PredResult(area='ALB', lat=42.65258, lon=-73.756233, lat_lon=(42.65258, -73.756233), grid_zone='CAPITL', ptid=61757.0, date=['2023-07-20', '2023-07-21', '2023-07-22'], max_temp=[29.2, 27.4, 26.0], min_temp=[16.7, 16.6, 14.6], predicted_load=None),
 PredResult(area='ART', lat=43.974785, lon=-75.910759, lat_lon=(43.974785, -75.910759), grid_zone='MHK VL', ptid=61756.0, date=['2023-07-20', '2023-07-21', '2023-07-22'], max_temp=[28.1, 23.8, 25.0], min_temp=[15.3, 15.3, 14.2], predicted_load=None),
 PredResult(area='BGM', lat=42.098843, lon=-75.920647, lat_lon=(42.098843, -75.920647), grid_zone='CENTRL', ptid=61754.0, date=['2023-07-20', '2023-07-21', '2023-07-22'], max_temp=[28.2, 24.6, 23.6], min_temp=[8.2, 14.1, 10.4], predicted_load=None),
 PredResult(area='BUF', lat=42.88023, lon=-78.878738, lat_lon=(42.88023, -78.878738), grid_zone='WEST', ptid=61752.0, date=['2023-07-20', '2023-07-21', '2023-07-22'], max_temp=[29.6, 24.5, 24.7], min_temp=[17.7, 17.1, 14.6], predicted_load=None),
 Pr